# DATA Extraction

In [1]:
import os
from collections import defaultdict
import sys
import time
import json
import psutil
import argparse
import operator
import pandas as pd

sys.path.append("/home/apis/nykaa")
from pas.v2.utils import Utils
sys.path.append('/home/ubuntu/nykaa_scripts/utils')
import pandasutils

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
nykaadb = Utils.redshiftConnection()
cursor = nykaadb.cursor()

Query for Extracting data with limit

In [ ]:
customer_orders_query_with_limit = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item sfoi INNER JOIN sales_flat_order sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfoi.parent_item_id IS NULL AND sfo.customer_id IS NOT NULL LIMIT 50000;"

In [ ]:
redshift_customer_orders_query_with_limit = "SELECT fact_order_new.nykaa_orderno, fact_order_new.order_customerid, fact_order_detail_new.product_id, fact_order_detail_new.product_sku, fact_order_detail_new.sku_type, fact_order_detail_new.product_name, fact_order_detail_new.order_date from fact_order_new INNER JOIN fact_order_detail_new ON fact_order_new.nykaa_orderno=fact_order_detail_new.nykaa_orderno WHERE fact_order_new.nykaa_orderno <> 0 AND product_mrp > 1 AND order_customerid IS NOT NULL AND order_date > '2018-01-01 00:00:00'"

Query for Extracting data without limit

In [ ]:
customer_orders_query_without_limit = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item sfoi INNER JOIN sales_flat_order sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfo.customer_id IS NOT NULL;"
#customer_orders_query_without_limit = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item sfoi INNER JOIN sales_flat_order sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfoi.parent_item_id IS NULL AND sfo.customer_id IS NOT NULL;"

In [3]:
redshift_customer_orders_query_without_limit = "SELECT fact_order_new.nykaa_orderno, fact_order_new.order_customerid, fact_order_detail_new.product_id, fact_order_detail_new.product_sku, fact_order_detail_new.sku_type, fact_order_detail_new.product_name, fact_order_new.order_date from fact_order_new INNER JOIN fact_order_detail_new ON fact_order_new.nykaa_orderno=fact_order_detail_new.nykaa_orderno WHERE fact_order_new.nykaa_orderno <> 0 AND product_mrp > 1 AND order_customerid IS NOT NULL AND order_date > '2018-06-01 00:00:00'"

Query for Extracting data from archive table

In [ ]:
customer_orders_archive_query = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item_archive sfoi INNER JOIN sales_flat_order_archive sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfo.customer_id IS NOT NULL;"
#customer_orders_archive_query = "SELECT sfoi.order_id, sfo.customer_id, sfoi.product_id, sfoi.product_type, sfoi.name, sfo.created_at FROM sales_flat_order_item_archive sfoi INNER JOIN sales_flat_order_archive sfo ON sfo.entity_id=sfoi.order_id WHERE sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfoi.parent_item_id IS NULL AND sfo.customer_id IS NOT NULL;"

Dataframe for storing extracted data

In [4]:
df = pd.DataFrame()

Extracting data without limits

In [5]:
rows = Utils.fetchResultsInBatch(Utils.redshiftConnection(), redshift_customer_orders_query_without_limit, 10000)
df = df.append(rows)
len(rows), len(df)

(25008792, 25008792)

In [6]:
df.head()

,0,1,2,3,4,5,6
0,505181114,4655221,82288.0,NYPAC00000214,SIMPLE,PAC Intense Auto Kohl Pencil,2018-06-01 00:01:52
1,505181114,4655221,19750.0,8903380157956,SIMPLE,Faces Magneteyes Kajal - Lasts All Day - Black,2018-06-01 00:01:52
2,505181114,4655221,82287.0,NYPAC00000212,SIMPLE,PAC Longlasting Kohl Pencil,2018-06-01 00:01:52
3,505181114,4655221,220212.0,6928820043042,CONFIG,Garnier Skin Naturals Micellar Cleansing Water,2018-06-01 00:01:52
4,505181106,6271562,197278.0,8901030649653,SIMPLE,Lakme Whitening Rose Powder With Sunscreen,2018-06-01 00:02:00


Extracting data with limits

In [ ]:
rows = Utils.fetchResultsInBatch(Utils.nykaaMysqlConnection(), customer_orders_query_with_limit, 10000)
df = df.append(rows)
len(rows), len(df)

Extracting archive data

In [ ]:
rows = Utils.fetchResultsInBatch(Utils.nykaaMysqlConnection(), customer_orders_archive_query, 10000)
df = df.append(rows)
len(rows), len(df)

In [ ]:
df.head()

In [7]:
df.columns = ['order_id', 'customer_id', 'product_id', 'sku', 'product_type', 'product_name', 'created_at']

In [8]:
df.head()

,order_id,customer_id,product_id,sku,product_type,product_name,created_at
0,505181114,4655221,82288.0,NYPAC00000214,SIMPLE,PAC Intense Auto Kohl Pencil,2018-06-01 00:01:52
1,505181114,4655221,19750.0,8903380157956,SIMPLE,Faces Magneteyes Kajal - Lasts All Day - Black,2018-06-01 00:01:52
2,505181114,4655221,82287.0,NYPAC00000212,SIMPLE,PAC Longlasting Kohl Pencil,2018-06-01 00:01:52
3,505181114,4655221,220212.0,6928820043042,CONFIG,Garnier Skin Naturals Micellar Cleansing Water,2018-06-01 00:01:52
4,505181106,6271562,197278.0,8901030649653,SIMPLE,Lakme Whitening Rose Powder With Sunscreen,2018-06-01 00:02:00


# Preparing child to parent dict for products

In [ ]:
query = "select child_id, parent_id from catalog_product_relation"

rows = Utils.fetchResultsInBatch(Utils.nykaaMysqlConnection(), query, 50000)
child_2_parent = {row[0]: row[1] for row in rows}

In [10]:
results = Utils.scrollESForResults()
child_2_parent = results['child_2_parent']
sku_2_product_id = results['sku_2_product_id']

In [12]:
with open("/home/ubuntu/data/cab/child_2_parent.json", "r+") as f:
    child_2_parent.update({int(key): int(value) for key, value in json.load(f).items()})

In [14]:
with open("/home/ubuntu/data/cab/sku_2_product_id.json", "r+") as f:
    sku_2_product_id.update({key: int(value) for key, value in json.load(f).items()})

In [16]:
%time
df['product_id'] = df.apply(lambda row: sku_2_product_id.get(row['sku'], row['product_id']), axis=1)
df['product_id'] = df.apply(lambda row: child_2_parent.get(row['product_id'], row['product_id']), axis=1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.44 µs


In [ ]:
def convert_sku_2_product_id(data):
    data['product_id'] = data.apply(lambda row: sku_2_product_id.get(row['sku'], row['product_id']), axis=1)
    return data

df = pandasutils.parallelize_dataframe(df, convert_sku_2_product_id, 10, 16)

In [ ]:
def convert_2_parent_product(data):
    data['product_id'] = data.apply(lambda row: child_2_parent.get(row['product_id'], row['product_id']), axis=1)
    return data

In [ ]:
df = pandasutils.parallelize_dataframe(df, convert_2_parent_product, 10, 16)

In [ ]:
df.product_id.nunique()

In [ ]:
#df = pd.read_csv('/home/ubuntu/data/cab/latest_7_jan.csv')

In [76]:
df = df[~df.order_id.isna() & ~df.customer_id.isna() & ~df.product_id.isna()]

In [82]:
df['product_id'] = df.product_id.values.astype(np.int64)

# Removing duplicate products from an order

In [ ]:
df = df.filter(['customer_id', 'order_id', 'product_id'])

In [ ]:
df['product_count'] = 1

In [ ]:
df = df.groupby(['customer_id', 'order_id', 'product_id']).agg({'product_count': 'count'}).reset_index()

In [ ]:
len(df)

In [ ]:
df = df.drop(['product_count'], axis=1)

In [ ]:
df.head()

# Adding is_luxe field to data

First extracting out all the luxe products

In [ ]:
import sys
sys.path.append("/nykaa/api")
from pas.v2.utils import Utils

unique_product_ids = list(map(lambda x: int(x), list(df['product_id'].unique())))
es_query = {
    "query": {
        "bool": {
            "must": [
                {"term": {"is_luxe": True}},
                {"terms": {"product_id.keyword": unique_product_ids}}
            ]
        }
    },
    "_source": ["product_id"]
}

es_conn = Utils.esConn()
scroll_id = None
luxe_products = []
while True:
    if not scroll_id:
        response = es_conn.search(index='livecore', body=es_query, scroll='25m')
    else:
        response = es_conn.scroll(scroll_id=scroll_id, scroll='25m')
        
    if not response['hits']['hits']:
        break
      
    scroll_id = response['_scroll_id']
    luxe_products += [int(hit['_source']['product_id']) for hit in response['hits']['hits']]
    
luxe_products_dict = {int(product_id): True for product_id in luxe_products}

In [ ]:
luxe_products_dict[10839]

Now adding the field extracted

In [ ]:
df['is_luxe'] = df['product_id'].apply(lambda x: luxe_products_dict[int(x)] if luxe_products_dict.get(int(x)) else False)

In [ ]:
df.head()

In [ ]:
df[df['is_luxe'] == True].head()

# Writing to CSV File

In [11]:
df.to_csv('/home/ubuntu/data/cab/cab_8_jan.csv', encoding='utf-8', index=False)

In [85]:
len(df)

41442835

In [86]:
df_2018 = df[df.created_at > '2018-01-01 00:00:00']

In [87]:
len(df_2018)

23327042

In [57]:
df_2018.head()

,order_id,customer_id,product_id,sku,product_type,product_name,created_at
1784660,503157588,650079,131991.0,8904086350146,SIMPLE,Vega Shampoo Comb 1268,2018-01-01 00:05:29
1784661,701066043,1867643,28012.0,8904245701291,CONFIG,Nykaa Floral Carnival Nail Enamel - Carefree C...,2018-01-01 00:07:57
1784662,701066043,1867643,61421.0,8904245700805,CONFIG,Nykaa Matte Nail Enamel - S'Mores Milkshake,2018-01-01 00:07:57
1784663,701066043,1867643,61421.0,8904245700911,CONFIG,Nykaa Matte Nail Enamel - Cherry Pop,2018-01-01 00:07:57
1784664,503157596,650079,131991.0,8904086350146,SIMPLE,Vega Shampoo Comb 1268,2018-01-01 00:08:07


In [89]:
df_2018.to_csv('/home/ubuntu/data/cab/raw_cab_2018_till_6_oct.csv', encoding='utf-8', index=False)

In [20]:
df = pd.read_csv('/home/ubuntu/data/cab/cab_7_jan.csv')

In [21]:
df.head()

,order_id,customer_id,product_id,sku,product_type,product_name,created_at
0,505181137,613295,188915.0,8901314308009,SIMPLE,Colgate Bubble Fruit Flavour Minions Toothpast...,2018-06-01 00:04:19
1,505181137,613295,13368.0,8906009450414,SIMPLE,Biotique Bio Kelp fresh growth Protein Shampoo,2018-06-01 00:04:19
2,505181137,613295,39968.0,8906009452616,SIMPLE,Biotique Bio Honey Gel Refreshing Foaming Face...,2018-06-01 00:04:19
3,505181137,613295,209020.0,8901030665066,SIMPLE,Dove Cream Beauty Bathing Soap (Buy 4 Get 1),2018-06-01 00:04:19
4,505181137,613295,213461.0,8901314306500,SIMPLE,Colgate Cibaca Vedshakti Toothpaste,2018-06-01 00:04:19


In [23]:
df.order_id.nunique(), df.product_id.nunique()

(7178232, 72582)

In [24]:
len(df)

24869502